# Imports

In [17]:
import numpy as np
import pandas as pd
import random
import heapq
import time

# Environment related Code

In [18]:
class DisasterZoneEnv:
    """
    A 2D grid environment to simulate a disaster zone for AI agents. The environment
    can be configured to be either static or dynamic, with the grid containing survivors,
    resources, obstacles, and a drone.

    Legend:
        0 -> Empty cell
        1 -> Obstacle
        2 -> Survivor
        3 -> Resource
        D -> Drone (tracked separately, but displayed in render)
    """

    def __init__(self, width=10, height=10, num_obstacles=5, num_survivors=3, num_resources=2, initial_energy=50, dynamic=False):
        """
        Initialize the environment with configurable dimensions and grid contents.

        :param width: Width of the grid.
        :param height: Height of the grid.
        :param num_obstacles: Number of obstacles to place in the grid.
        :param num_survivors: Number of survivors to place in the grid.
        :param num_resources: Number of resources to place in the grid.
        :param initial_energy: Initial energy level of the drone.
        :param dynamic: Whether the environment is dynamic (changes during simulation).
        """
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic  # Determines whether the environment changes during simulation
        self.reset()

    def reset(self):
        """
        Resets the environment to its initial state by randomly placing obstacles,
        survivors, resources, and the drone.

        :return: The initial state of the drone (position, energy).
        """
        # Initialize an empty grid
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Place obstacles, survivors, and resources randomly
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1  # Obstacle
        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2  # Survivor
        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3  # Resource

        # Place the drone at a random position
        self.drone_x, self.drone_y = self._get_random_empty_cell()

        # Set the drone's initial energy level
        self.energy = self.initial_energy

        # Return the initial state (drone position and energy)
        return self._get_state()

    def _get_random_empty_cell(self):
        """
        Finds a random empty cell in the grid that is not occupied by any obstacles,
        survivors, resources, or the drone.

        :return: Coordinates (x, y) of an empty cell.
        """
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:  # Ensure the cell is empty
                return x, y

    def _get_state(self):
        """
        Returns the current state of the drone.

        :return: Tuple containing the drone's position (x, y) and remaining energy.
        """
        return (self.drone_x, self.drone_y, self.energy)

    def step(self, x, y):
        """
        Moves the drone to a specific position (x, y) on the grid.

        :param x: Target x-coordinate.
        :param y: Target y-coordinate.
        """
        self.drone_x, self.drone_y = x, y
        self.energy -= 1  # Deduct energy for the move

    def apply_dynamic_changes(self, step_count):
        """
        Applies dynamic changes to the grid, such as adding obstacles, moving survivors,
        and placing new resources, based on the current step count.

        :param step_count: The current simulation step.
        """
        if self.dynamic:
            # Add a new obstacle every 5 steps
            if step_count % 5 == 0:
                x, y = self._get_random_empty_cell()
                self.grid[x, y] = 1  # Add an obstacle
                print(f"Dynamic Change: Added obstacle at ({x}, {y})")

            # Move survivors every 3 steps
            if step_count % 3 == 0:
                survivor_positions = [(x, y) for x in range(self.height)
                                      for y in range(self.width) if self.grid[x, y] == 2]
                for x, y in survivor_positions:
                    self.grid[x, y] = 0  # Remove survivor from the current position
                    new_x, new_y = self._get_random_empty_cell()
                    self.grid[new_x, new_y] = 2  # Place survivor in a new position
                    print(f"Dynamic Change: Moved survivor from ({x}, {y}) to ({new_x}, {new_y})")

            # Add a new resource every 7 steps
            if step_count % 7 == 0:
                x, y = self._get_random_empty_cell()
                self.grid[x, y] = 3  # Add a resource
                print(f"Dynamic Change: Added resource at ({x}, {y})")

    def render(self):
        """
        Renders the current state of the grid by printing it to the console.
        """
        # Copy the grid for visualization
        grid_copy = np.copy(self.grid).astype(str)

        # Replace numeric values with symbols for better readability
        grid_copy[grid_copy == '0'] = '.'  # Empty cell
        grid_copy[grid_copy == '1'] = '#'  # Obstacle
        grid_copy[grid_copy == '2'] = 'S'  # Survivor
        grid_copy[grid_copy == '3'] = 'R'  # Resource

        # Place the drone in the visualization
        grid_copy[self.drone_x, self.drone_y] = 'D'

        # Print the grid row by row
        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")

# Dijikstra Agent Related Code

In [19]:
class DijkstraAgent:
    """
    An agent that uses Dijkstra's algorithm to find the shortest path
    to the nearest target (e.g., survivor or resource) in the DisasterZoneEnv.
    """

    def __init__(self, env):
        """
        Initializes the agent with the environment.

        :param env: An instance of the DisasterZoneEnv class.
        """
        self.env = env

    def dijkstra(self, grid, start, target):
        """
        Dijkstra's algorithm to find the shortest path in a 2D grid.

        :param grid: The grid representing the environment.
        :param start: The starting position of the agent (x, y).
        :param target: The target position (x, y).
        :return: The shortest path as a list of coordinates and the total distance.
        """
        # Movement directions (up, down, left, right)
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        # Priority queue to store distances and positions
        pq = [(0, start)]
        distances = {start: 0}
        previous = {start: None}

        while pq:
            # Get the node with the smallest distance
            current_distance, current_position = heapq.heappop(pq)

            if current_position == target:
                # Reconstruct the path from start to target
                path = []
                while current_position is not None:
                    path.append(current_position)
                    current_position = previous[current_position]
                path.reverse()
                return path, current_distance

            # Explore neighbors
            for dx, dy in directions:
                neighbor = (current_position[0] + dx, current_position[1] + dy)

                if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1] and grid[neighbor[0], neighbor[1]] != 1:
                    new_distance = current_distance + 1
                    if neighbor not in distances or new_distance < distances[neighbor]:
                        distances[neighbor] = new_distance
                        previous[neighbor] = current_position
                        heapq.heappush(pq, (new_distance, neighbor))

        return [], float('inf')  # Return an empty path if the target is unreachable

    def find_closest_target(self, target_type):
        """
        Finds the closest target of a given type (survivor or resource).

        :param target_type: The type of target to find (2 for survivor, 3 for resource).
        :return: The shortest path to the closest target and the target position.
        """
        start = (self.env.drone_x, self.env.drone_y)
        targets = [(x, y) for x in range(self.env.height) for y in range(self.env.width) if self.env.grid[x, y] == target_type]

        if not targets:
            return None, None, float('inf')  # No targets available

        shortest_path, closest_target, shortest_distance = None, None, float('inf')
        for target in targets:
            path, distance = self.dijkstra(self.env.grid, start, target)
            if path and distance < shortest_distance:  # Ensure path exists
                shortest_path, closest_target, shortest_distance = path, target, distance

        # If no reachable targets, return None
        if shortest_distance == float('inf'):
            print(f"No reachable targets of type {target_type}.")
            return None, None, float('inf')

        return shortest_path, closest_target, shortest_distance



    def execute(self, dynamic=False, step_limit=500):
        """
        The main loop of the Dijkstra Agent. Finds and moves to targets (survivors and resources)
        until the agent runs out of energy, there are no more targets, or a step limit is reached.

        :param dynamic: Whether to apply dynamic changes to the environment.
        :param step_limit: The maximum number of steps before stopping.
        :return: Total number of cells explored during the simulation.
        """
        step_count = 0
        self.cells_explored = 0  # Reset cells explored counter

        while self.env.energy > 0 and step_count < step_limit:
            if dynamic:
                self.env.apply_dynamic_changes(step_count)

            # Find the closest survivor first, then resources
            path, target_position, distance = self.find_closest_target(2)  # Try to find survivors
            if not path:
                path, target_position, distance = self.find_closest_target(3)  # Try to find resources

            # If no targets are reachable, stop the simulation
            if not path:
                print("No reachable targets remaining. Stopping simulation.")
                break

            # Move the drone along the computed path
            for step in path[1:]:  # Skip the starting position
                self.env.step(*step)
                step_count += 1

                # Optional: Only render every 5 steps for better performance
                if step_count % 5 == 0:
                    self.env.render()

                if dynamic:
                    self.env.apply_dynamic_changes(step_count)

                # Stop if energy is exhausted
                if self.env.energy <= 0:
                    print("Drone ran out of energy!")
                    break

            # Stop if step limit is reached
            if step_count >= step_limit:
                print("Step limit reached. Stopping simulation.")
                break

        print("Agent has completed its task.")
        return self.cells_explored  # Return total cells explored




# Tester Related Code

In [20]:
class Tester:
    """
    A generalized testing framework for evaluating AI agents in the DisasterZoneEnv.
    """

    def __init__(self, env, num_tests=5, verbose=True):
        """
        Initialize the testing framework.

        :param env: The DisasterZoneEnv instance.
        :param num_tests: Number of test runs.
        :param verbose: Whether to print detailed logs.
        """
        self.env = env
        self.num_tests = num_tests
        self.verbose = verbose
        self.results = []  # Store results for all tests

    def run(self, agent_class, dynamic=False, step_limit=500):
        """
        Run tests for the specified agent and record metrics.

        :param agent_class: The agent class to test (e.g., DijkstraAgent).
        :param dynamic: Whether to enable dynamic changes in the environment.
        :param step_limit: Maximum steps per test.
        """
        for test_idx in range(self.num_tests):
            # Reset the environment for each test
            self.env.reset()
            agent = agent_class(self.env)

            if self.verbose:
                print(f"\n=== Test {test_idx + 1}/{self.num_tests} ===")
            
            # Start tracking metrics
            initial_survivors = np.sum(self.env.grid == 2)  # Count initial survivors
            initial_resources = np.sum(self.env.grid == 3)  # Count initial resources
            initial_energy = self.env.energy

            # Run the agent in the environment
            steps_explored = agent.execute(dynamic=dynamic, step_limit=step_limit)

            # Calculate results
            survivors_left = np.sum(self.env.grid == 2)  # Survivors left
            resources_left = np.sum(self.env.grid == 3)  # Resources left

            survivors_rescued = initial_survivors - survivors_left
            resources_collected = initial_resources - resources_left
            energy_used = initial_energy - self.env.energy

            # Add test results to the results list
            self.results.append({
                "Test": test_idx + 1,
                "Dynamic": dynamic,
                "Survivors Rescued": survivors_rescued,
                "Resources Collected": resources_collected,
                "Energy Used": energy_used,
                "Steps Explored": steps_explored,
            })

            # Print results for the test if verbose is enabled
            if self.verbose:
                print(f"Survivors Rescued: {survivors_rescued}")
                print(f"Resources Collected: {resources_collected}")
                print(f"Energy Used: {energy_used}")
                print(f"Steps Explored: {steps_explored}")


    def display_results(self):
        """
        Displays the collected test results in a table format using pandas.
        """
        if not self.results:
            print("No results to display.")
            return

        # Convert results to a pandas DataFrame for tabular output
        df = pd.DataFrame(self.results)
        print("\n=== Results Table ===")
        print(df)
        return df


# Main Code for Runinng Tests

In [ ]:
if __name__ == "__main__":
    # Static Environment Test
    print("\n=== STATIC ENVIRONMENT TEST ===")
    static_env = DisasterZoneEnv(
        width=10, height=10, 
        num_obstacles=8, num_survivors=5, num_resources=2, 
        initial_energy=20, dynamic=False
    )
    static_tester = Tester(static_env, num_tests=3, verbose=True)
    static_tester.run(agent_class=DijkstraAgent, dynamic=False)
    static_results = static_tester.display_results()


=== STATIC ENVIRONMENT TEST ===

=== Test 1/3 ===
